# Getting Ticket Information from ServiceNow using Python & requests

This is a tutorial for getting access to incident tickets from the UCSC ServiceNow system. This should be applicable to other ServiceNow installations, but this is not guaranteed. Use at your own risk!

### Goals: Download ticket information from the UCSC ServiceNow system and do some basic analysis
### Tools: Python & requests

Python (specifically Python 3) is a scripting language that is simple to use and incredibly powerful. I won't teach all of Python in this tutorial, but it is a relatively easy language to pick up. Some resources I would recommend are listed below: 

[Python Guru - Getting Started with Python](https://thepythonguru.com/getting-started-with-python/)

[LearnPython.org](https://www.learnpython.org/)

[Learning Python Overview](https://docs.python-guide.org/intro/learning/)

It will work for our purposes, since we're going to be sending a request for data to the ServiceNow API and then downloading it for analysis later.

requests is a Python module used for sending HTTP requests. This is commonly used for sending requests to APIs (Application Programming Interfaces), which, fortunately, is exactly what we're using it for. 

## Sending a Request
To send a request, we first need a URL to send the request to (this is called an endpoint). This URL is modified and can be added to in order to request different information from the API. In our case, the API endpoint for ServiceNow incidents is:

https://ucsc.service-now.com/api/now/table/incident?

In our case, we want to store this as a Python variable, which can be done as follows:

In [5]:
url = 'https://ucsc.service-now.com/api/now/table/incident?'

The idea with this endpoint is to concatenate our parameters to the end of this url, and then send a request to it. This will (hopefully, if everything works) return tickets that match our parameters.

## Setting up filters
If we want to retrieve a specific set of tickets, we can use ServiceNow filters. To do this, navigate to your ServiceNow instance and set up a filter of tickets that you would like to look at. Then, right click the blue text that defines the filter (make sure you select the parameter that is the furthest to the right) and select 'Copy query'. The text copied should look something like this:

active=true^assigned_to=67c139b309641440fa07e749fee81bd7^incident_state=1

This string defines ticket filter itself. If you look closely, you will see that it is broken into various conditions, separated by ^ characters. These are considered ANDs, meaning each of the conditions must be true for a ticket to be caught in the filter. Each condition is listed below:

active=true

assigned_to=67c139b309641440fa07e749fee81bd7

incident_state=1

As we can see, this filter finds tickets that are active, assigned to a specific department, and the incident's state is active. For this filter to work with our request to the ServiceNow API, we need to add 'sysparm_query=' to the front, which basically tells ServiceNow that the we are querying their system. Our resulting filter string looks like this:

In [15]:
filter_str = 'sysparm_query=active=true^assigned_to=67c139b309641440fa07e749fee81bd7^incident_state=1'

## Setting headers
Before we send out a request, we need to provide some extra information to the ServiceNow API. This is done with a header, which is defined as follows:

In [2]:
headers = {'Content-Type':'application/json','Accept':'application/json'}

This header tells ServiceNow that we want JSON (JavaScript Object Notation) data as a response to our request. Python can handle JSON data very nicely, which makes it a good fit for processing our resulting data.

## Sending the request
Now we have all the pieces necessary to send a request for the tickets we want. To recap, we needed:

- An endpoint URL (where the request is sent)
- A filter string (what tickets we want)
- Headers (telling ServiceNow what sort of data we want)
- Authentication (an account that sends the request)

Now, to make our actual request, we use the aptly named requests library. The request module has to be imported before we use it, however.

In [3]:
import requests

We need to make the full URL we are sending a request to. This is done by combining (concatenating) the endpoint URL with our filter to form the request url.

In [16]:
request_url = url  + filter_str

To make sure our code is able to handle errors, we define our tickets object to be None at the start. Then, after the request, we can set the tickets object to be the response. This way, if we have an error, we can handle it and find the source of the issue.

In [21]:
tickets = None

Now we can actually send the request. Since we are getting data from the API, we send a GET request using the .get() function from the requests module. Here it goes:

In [22]:
response = requests.get(request_url, auth=('your_username','your_password'), headers=headers)

Now we've got a response, but we should check if any error has occured. The response has a handy field denoting the status code that the response got. If an HTTP response was sent and recieved a status code of 200, that means the request was successful. We want to be alerted if the response status code was anything but 200, so we can write a condition to do so.

In [23]:
if response.status_code != 200:
    raise ConnectionError('Error getting tickets, recieved status code: {}'.format(response.status_code))
else:
    tickets = response.json()

Once we do get a 200 response, we get the JSON associated with the response with the .json() call. Since the resulting object can be treated like a dictionary, we can look inside and see if there are any keys we want to look at.

In [29]:
tickets.keys()
tickets['result'][0].keys()

dict_keys(['parent', 'caused_by', 'u_resolution', 'watch_list', 'upon_reject', 'sys_updated_on', 'approval_history', 'skills', 'u_computer_os', 'u_guest_email', 'number', 'u_security_type', 'state', 'sys_created_by', 'u_computer_mac_address', 'knowledge', 'order', 'u_remove_client_filter', 'u_subcategory', 'cmdb_ci', 'delivery_plan', 'impact', 'active', 'priority', 'business_duration', 'group_list', 'u_long_resolution', 'short_description', 'u_product', 'u_prev_assignmnt_grp', 'correlation_display', 'work_start', 'additional_assignee_list', 'u_delay_due_date', 'notify', 'sys_class_name', 'closed_by', 'follow_up', 'u_priority', 'u_signed_ati', 'reopened_by', 'reassignment_count', 'assigned_to', 'u_assigned_to', 'u_resnet_topic', 'sla_due', 'u_actual_problem', 'u_websvc_topic', 'u_milestone', 'u_jack_number', 'u_category', 'escalation', 'upon_approval', 'correlation_id', 'u_research_resolution', 'u_client_defined_end_date', 'u_sla_due', 'u_filter_applications', 'u_first_contact', 'made_s

We now have tickets in a list! They are stored within the 'result' key within the JSON response. Since multiple tickets matched the filter, the result key can be iterated through as follows:

In [28]:
for ticket in tickets['result']:
    print(ticket['short_description'])

Clone hard drive into a different computers hard drive


The rest is up to you! Each ticket has contains keys for different ticket attributes. Some may be named in ways you may not expect ('caller_id' corresponds to client information), but with a bit of digging, all necessary ticket information can be found. 

Note: Ticket comments and client information is stored in what's known as a journal, which is not directly stored with the ticket. It requires an entirely new HTTP request with the 'sys_id' of the ticket/client in question. More information can be found here:

https://www.servicelater.co.uk/retrieving-comments-journals-from-a-ticket-using-the-servicenow-api/